In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import paho.mqtt.client as mqtt

In [ ]:
# Callback function when a message is received
def on_message(client, userdata, message):
    # Decode the message payload and print it
    print(f"Message received: {message.payload.decode()} on topic {message.topic}")

# Define the MQTT broker details
broker = "mqtt.119.ovh"  # Your broker address
port = 1883  # Default MQTT port for non-SSL
topic = "diya07/rx"  # The topic you're subscribing to
keep_alive = 60  # Keep alive time in seconds

# Create an MQTT client instance
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)

# Assign the on_message callback
client.on_message = on_message

# Connect to the broker
client.connect(broker, port, keep_alive)

# Subscribe to the specified topic
client.subscribe(topic)

# Loop forever to wait for messages
client.loop_forever()


In [ ]:
import paho.mqtt.client as mqtt
import time

# Define the MQTT broker details
broker = "mqtt.119.ovh"  # Your broker address
port = 1883  # Default MQTT port for non-SSL
topic = "diya07"
read_topic = topic + "/rx"
transmit_topic = topic + "/tx"
keep_alive = 60

# Global variable to store the last received sensor data
last_sensor_data = None

# Callback function to handle received messages (sensor data)
def on_message(client, userdata, message):
    global last_sensor_data
    # Decode the received sensor data
    last_sensor_data = float(message.payload.decode())  # Assuming sensor data is a float
    print(f"Received sensor data: {last_sensor_data}")

# Function to simulate control calculation
def compute_control_input(sensor_data):
    # Perform some computation (e.g., control logic)
    # In this example, we'll just double the sensor value as a placeholder for the control calculation
    control_input = sensor_data * 2
    return control_input

# Function to transmit the control input via MQTT
def send_control_input(client, control_input):
    # Publish the control input to the specified output topic
    client.publish(transmit_topic, str(control_input))
    print(f"Sent control input: {control_input}")

# Create an MQTT client instance
client = mqtt.Client()

# Assign the on_message callback
client.on_message = on_message

# Connect to the broker
client.connect(broker, port, keep_alive)

# Subscribe to the sensor data topic
client.subscribe(read_topic)

# Start the network loop in the background
client.loop_start()

# Control loop
while True:
    # Wait until we receive some sensor data
    if last_sensor_data is not None:
        # Step 1: Perform the calculation (MPC or some other control logic)
        control_input = compute_control_input(last_sensor_data)
        
        # Step 2: Send the control input back via MQTT
        send_control_input(client, control_input)
        
        # Reset last_sensor_data to None to wait for the next measurement
        last_sensor_data = None
    
    # Sleep for a short duration before checking again
    time.sleep(1)

# Stop the network loop and disconnect the client when done (not reached in this loop)
client.loop_stop()
client.disconnect()
